<a href="https://colab.research.google.com/github/sa-y-an/OMR_Net/blob/main/Final_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing stuffs
import numpy as np
import gc 
import os
import ctypes
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import LeakyReLU
import pandas as pd

sns.set()

custom activation functions

In [ ]:
# pflu activation function
def pflu_activation(x):
    return x*0.5*(1+(x/(1+x**2)**0.5))

#fpflu actiavtion function
def fpflu_activation(x):
    return tf.math.maximum(x,x/(1+x**2)) 

Fetching dataset from github

In [ ]:
!wget https://github.com/sa-y-an/answersheet_data/raw/main/folds.zip 

In [ ]:
from zipfile import ZipFile

In [ ]:
file_name = "folds.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    #zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

Preparing the dataset

In [ ]:
#preparing dataset

import pathlib
data_dir1 = '/content/folds/1'
data_dir2 = '/content/folds/2'
data_dir3 = '/content/folds/3'
data_dir4 = '/content/folds/4'
data_dir5 = '/content/folds/5'

data_dir1 = pathlib.Path(data_dir1)
data_dir2 = pathlib.Path(data_dir2)
data_dir3 = pathlib.Path(data_dir3)
data_dir4 = pathlib.Path(data_dir4)
data_dir5 = pathlib.Path(data_dir5)


batch_size = 32
img_height = 224
img_width = 224

# fetching data from directory

img_ds1=tf.keras.preprocessing.image_dataset_from_directory(
        data_dir1,
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)

img_ds2=tf.keras.preprocessing.image_dataset_from_directory(
        data_dir2,
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)

img_ds3=tf.keras.preprocessing.image_dataset_from_directory(
        data_dir3,
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)
 
img_ds4=tf.keras.preprocessing.image_dataset_from_directory(
        data_dir4,
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)

img_ds5=tf.keras.preprocessing.image_dataset_from_directory(
        data_dir5,
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size) 

# normalizing all the images to a float btw 0 , 2555

def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

img_ds1 = img_ds1.map(process)
img_ds2 = img_ds2.map(process)
img_ds3 = img_ds3.map(process)
img_ds4 = img_ds4.map(process)
img_ds5 = img_ds5.map(process)

Uncomment that model which you will use to save memory space

In [ ]:
# transfer learning model 
# mobile net V2 model used

feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)


In [ ]:
classification_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

pretrained_model_without_top_layer_class = hub.KerasLayer(
    classification_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
# Dense Net 201 Model

# base_model_dn201 = tf.keras.applications.DenseNet201(
#     include_top=True,
#     weights="imagenet")


In [ ]:
# VGG19 Model

# base_model_vgg19 = tf.keras.applications.VGG19(
#     include_top=True,
#     weights="imagenet"
# )


In [ ]:
# Functions 

# chosing the correct fold
def create_dataset(i) :    
    
    if i == 0 :
        train_dsu = img_ds2.concatenate(img_ds3.concatenate(img_ds4.concatenate(img_ds5)))
        test_dsu = img_ds1
    if i == 1 :
        train_dsu = img_ds1.concatenate(img_ds3.concatenate(img_ds4.concatenate(img_ds5)))
        test_dsu = img_ds2
    if i == 2 :
        train_dsu = img_ds1.concatenate(img_ds2.concatenate(img_ds4.concatenate(img_ds5)))
        test_dsu = img_ds3
    if i == 3 :
        train_dsu = img_ds1.concatenate(img_ds2.concatenate(img_ds3.concatenate(img_ds5)))
        test_dsu = img_ds4
    if i == 4 :
        train_dsu = img_ds1.concatenate(img_ds2.concatenate(img_ds3.concatenate(img_ds4)))
        test_dsu = img_ds5  
        
    return (train_dsu,test_dsu)


# tuning and caching the data for faster training
def atune(train_dsu,test_dsu):
    
    AUTOTUNE = tf.data.AUTOTUNE

    train_dsu = train_dsu.cache().prefetch(buffer_size=AUTOTUNE)
    test_dsu = test_dsu.cache().prefetch(buffer_size=AUTOTUNE)
    
    return (train_dsu,test_dsu)



# division  of the folds into train test
def train_test_val(train_dsu):

    DATASET_SIZE = tf.data.experimental.cardinality(train_dsu).numpy()
    train_size = int(0.8 * DATASET_SIZE)
    val_size = int(0.2 * DATASET_SIZE)

    train_dsu = train_dsu.shuffle(DATASET_SIZE)
    train_dataset = train_dsu.take(train_size)
    val_dataset = train_dsu.skip(train_size)
    
    return (train_dataset,val_dataset)


In [ ]:
def create_model_mobnetV2_class() :
    
    num_classes = 3
    
    omr_model = tf.keras.Sequential([
      pretrained_model_without_top_layer_class,
      layers.Dense(640, activation= 'relu'),   
      layers.Dropout(0.25),
      layers.Dense(320,activation= fpflu_activation ),
      layers.Dropout(0.25),
      layers.Dense(160,activation= pflu_activation ),
      layers.Dropout(0.25),
      layers.Dense(num_classes,activation='softmax'),
    ])

    tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)
    
    
    omr_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
#     print(omr_model.summary())
    
    return omr_model

In [ ]:
# tf learning on mobnet V2 Model
def create_model_mobnetV2() :
    
    num_classes = 3
    
    omr_model = tf.keras.Sequential([
      pretrained_model_without_top_layer,
      layers.Dense(640, activation= 'relu'),   
      layers.Dropout(0.25),
      layers.Dense(320,activation= fpflu_activation ),
      layers.Dropout(0.25),
      layers.Dense(160,activation= pflu_activation ),
      layers.Dropout(0.25),
      layers.Dense(num_classes,activation='softmax'),
    ])

    tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)
    
    
    omr_model.compile(optimizer='RMSprop',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
#     print(omr_model.summary())
    
    return omr_model

In [ ]:
# tf learning on densenet 201
def create_dense_net() :


    num_classes = 3


    # Freeze base model
    base_model_dn201.trainable = False



    omr_model = tf.keras.Sequential([
      base_model_dn201,
      layers.Dense(640, activation= 'relu'),   
      layers.Dropout(0.2),
      layers.Dense(320,activation= fpflu_activation ),
      layers.Dropout(0.2),
      layers.Dense(160,activation= pflu_activation ),
      layers.Dropout(0.2),
      layers.Dense(num_classes,activation='softmax'),
    ])


    omr_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    # print(omr_model.summary())
    
    return omr_model

In [ ]:
# tf learning on densenet 201
def create_vgg19() :


    num_classes = 3


    # Freeze base model
    base_model_vgg19.trainable = False



    omr_model = tf.keras.Sequential([
      base_model_vgg19,
      layers.Dense(640, activation= 'relu'),   
      layers.Dropout(0.2),
      layers.Dense(320,activation= fpflu_activation ),
      layers.Dropout(0.2),
      layers.Dense(160,activation= pflu_activation ),
      layers.Dropout(0.2),
      layers.Dense(num_classes,activation='softmax'),
    ])


    omr_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    # print(omr_model.summary())
    
    return omr_model

In [ ]:
all_models = []

In [ ]:
# function used for iteratively training the model on folds

def model_training(epochs) :
    
    # vald_acc and test_d acc list
    vald_acc = []
    testd_acc = []

    
    for fold in range (5):

        train_dsu , test_dsu = create_dataset(fold)
        train_dsu , test_dsu = atune(train_dsu,test_dsu)
        train_dataset,val_dataset = train_test_val(train_dsu)

        del train_dsu



        print( str('This is for fold - ') + str(fold+1))

        
        #set the model type here
        model = create_model_mobnetV2()

        tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/{}".format(fold), histogram_freq=1)


        hv = model.fit(train_dataset,validation_data=val_dataset,epochs=epochs ,callbacks = [tb_callback] )
        vald_acc.append(hv)
        
        print('\n')

        gc.collect() # ggarbage collection

        ht = model.evaluate(test_dsu)       
        testd_acc.append(ht)

        # appending all models for future reference
        all_models.append(model)


        print('\n')
        print('\n')

        del test_dsu, train_dataset,val_dataset, model , ht , hv # deleting variables to clear memory
        
        libc = ctypes.CDLL("libc.so.6") # clearing cache 
        libc.malloc_trim(0)

        gc.collect() # garbage collection
        
        
    return vald_acc,testd_acc

In [ ]:
# set the number of epochs 
epochs = 10

In [ ]:
vald_acc,testd_acc = model_training(epochs)

In [ ]:
l = []
for i in testd_acc :
    l.append(i[1])

    
print(l)


In [ ]:
avg_val_acc = 0
for i in range(5) :
    avg_val_acc +=  vald_acc[i].history['val_accuracy'][epochs-1]
    
print(avg_val_acc/5)


In [ ]:
d = {'fold1' : [l[0]], 'fold2' : [l[1]] , 'fold3' : [l[2]] , 'fold4' : [l[3]] , 'fold5' : [l[4]] , 'fold_avg' : [np.mean(testd_acc, axis = 0)[1]] , 'vald_acc' : [avg_val_acc/5]}
df = pd.DataFrame(data=d)
df


In [ ]:
np.mean(testd_acc, axis = 0)[1]*100

Loading Tensorboard for analysis of results

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs

Downloading Models that give good accuracy 

In [ ]:
model2 = all_models[1]

In [ ]:
model2.save('fold2_95_88.h5')

In [ ]:
from google.colab import files
files.download('fold2_95_88.h5')